In [ ]:
import utils
import plotting
import postprocessing
import corrections

from utils import CUT_MAX_VAL
from hh_vars import years, sig_key, data_key, qcd_key, bg_keys, samples, txbb_wps
from postprocessing import (
    shape_var,
    shape_bins,
    blind_window,
    selection_regions,
    selection_regions_label,
    # selection_regions_year,
)

import numpy as np
import pandas as pd
from pandas.errors import SettingWithCopyWarning
from hist import Hist

import os
from copy import deepcopy
from inspect import cleandoc
import warnings

import matplotlib.pyplot as plt
import mplhep as hep
import matplotlib.ticker as mticker

plt.style.use(hep.style.CMS)
hep.style.use("CMS")
formatter = mticker.ScalarFormatter(useMathText=True)
formatter.set_powerlimits((-3, 3))
plt.rcParams.update({"font.size": 16})

# ignore these because they don't seem to apply
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_DIR = "../../../"
samples_dir = f"{MAIN_DIR}/../data/skimmer/Feb20"
signal_samples_dir = f"{MAIN_DIR}/../data/skimmer/Feb20"
year = "2017"

plot_dir = "../../../plots/PostProcessing/Feb21"
templates_dir = "templates/Feb21/"
_ = os.system(f"mkdir -p {plot_dir}")
_ = os.system(f"mkdir -p {templates_dir}")

systematics = {}

Load samples

In [ ]:
filters = None

# save cutflow as pandas table
cutflow = pd.DataFrame(index=list(samples.keys()))

# utils.remove_empty_parquets(samples_dir, year)
events_dict = utils.load_samples(signal_samples_dir, {sig_key: samples[sig_key]}, year, filters)
events_dict |= utils.load_samples(
    samples_dir, {k: samples[k] for k in samples.keys() - [sig_key]}, year, filters
)

utils.add_to_cutflow(events_dict, "BDTPreselection", "weight", cutflow)

# print weighted sample yields
for sample in events_dict:
    tot_weight = np.sum(events_dict[sample]["weight"].values)
    print(f"Pre-selection {sample} yield: {tot_weight:.2f}")

Scale factors and bb VV assignment

In [ ]:
postprocessing.apply_weights(events_dict, year, cutflow)
bb_masks = postprocessing.bb_VV_assignment(events_dict)
events_dict[sig_key] = postprocessing.postprocess_lpsfs(events_dict[sig_key])
cutflow

In [ ]:
postprocessing.load_bdt_preds(
    events_dict, year, samples_dir, list(samples.keys()), jec_jmsr_shifts=True
)

Overall BDT SF

In [ ]:
sel, cf = utils.make_selection(
    selection_regions[year]["BDTOnly"], events_dict, bb_masks, prev_cutflow=cutflow
)
lp_sf, unc, uncs = postprocessing.get_lpsf(events_dict[sig_key], sel[sig_key])
print(f"BDT Scale Factor: {lp_sf:.2f} ± {unc:.2f}")
print(uncs)
systematics["lp_sf_unc"] = unc / lp_sf
cf

Templates

In [ ]:
weight_shifts = {
    "pileup": [sig_key] + bg_keys,
    "PDFalphaS": [sig_key],
    "ISRPartonShower": [sig_key, "V+Jets"],
    "FSRPartonShower": [sig_key, "V+Jets"],
}

weight_labels = {
    "pileup": "Pileup",
    "PDFalphaS": "PDF",
    "ISRPartonShower": "ISR Parton Shower",
    "FSRPartonShower": "FSR Parton Shower",
}

# Check for 0 weights - would be an issue for weight shifts
print(
    "Any 0 weights:",
    np.any(
        [
            np.any(events["weight_nonorm"] == 0)
            for key, events in events_dict.items()
            if key != data_key
        ]
    ),
)

In [ ]:
prev_cutflow = cutflow
weight_key = "finalWeight"
cutstr = ""
var = shape_var[0]
show = True

for label, region in selection_regions[year].items():
    # print(region)
    pass_region = label.startswith("pass")

    if region == "BDTOnly":
        continue

    # if not pass_region:
    #     continue

    sel, cf = utils.make_selection(region, events_dict, bb_masks, prev_cutflow=prev_cutflow)

    # ParticleNetMD Txbb SFs
    sig_events = deepcopy(events_dict[sig_key][sel[sig_key]])
    sig_bb_mask = bb_masks[sig_key][sel[sig_key]]
    if pass_region:
        corrections.apply_txbb_sfs(sig_events, sig_bb_mask, year, weight_key)

    # set up samples
    hist_samples = list(events_dict.keys())
    for shift in ["down", "up"]:
        if pass_region:
            hist_samples.append(f"{sig_key}_txbb_{shift}")

        for wshift, wsamples in weight_shifts.items():
            for wsample in wsamples:
                hist_samples.append(f"{wsample}_{wshift}_{shift}")

    h = (
        Hist.new.StrCat(hist_samples, name="Sample")
        .Reg(*shape_bins, name=var, label=shape_var[1])
        .Weight()
    )

    for sample in events_dict:
        # print(sample)
        events = sig_events if sample == sig_key else events_dict[sample][sel[sample]]

        bb_mask = bb_masks[sample][sel[sample]]
        fill_data = {var: utils.get_feat(events, var, bb_mask)}
        weight = events[weight_key].values.squeeze()
        h.fill(Sample=sample, **fill_data, weight=weight)

        # add weight variations
        for wshift, wsamples in weight_shifts.items():
            if sample in wsamples:
                # print(wshift)
                for skey, shift in [("Down", "down"), ("Up", "up")]:
                    # reweight based on diff between up/down and nominal weights
                    sweight = (
                        weight
                        * (
                            events[f"weight_{wshift}{skey}"][0] / events["weight_nonorm"]
                        ).values.squeeze()
                    )
                    # print(weight)
                    # print(sweight)
                    # print(events[f"weight_{wshift}{skey}"][0])
                    # print(events["weight_nonorm"])
                    h.fill(Sample=f"{sample}_{wshift}_{shift}", **fill_data, weight=sweight)

    if pass_region:
        # blind Higgs mass window in pass region in data
        utils.blindBins(h, blind_window, data_key)

    if pass_region:
        # ParticleNetMD Txbb SFs
        fill_data = {var: utils.get_feat(sig_events, var, sig_bb_mask)}
        for shift in ["down", "up"]:
            h.fill(
                Sample=f"{sig_key}_txbb_{shift}",
                **fill_data,
                weight=sig_events[f"{weight_key}_txbb_{shift}"],
            )

    sig_scale = utils.getSignalPlotScaleFactor(events_dict, selection=sel)

    if plot_dir != "":
        plot_params = {"hists": h, "bg_keys": bg_keys, "sig_scale": sig_scale / 2, "show": show}
        plotting.ratioHistPlot(
            **plot_params,
            title=f"{selection_regions_label[label]} Region Pre-Fit Shapes",
            name=f"{plot_dir}/{cutstr}{label}_region_bb_mass.pdf",
        )

        for wshift, wsamples in weight_shifts.items():
            wlabel = weight_labels[wshift]

            if wsamples == [sig_key]:
                plotting.ratioHistPlot(
                    **plot_params,
                    sig_err=wshift,
                    title=f"{selection_regions_label[label]} Region {wlabel} Unc. Shapes",
                    name=f"{plot_dir}/{cutstr}{label}_region_bb_mass_{wshift}.pdf",
                )
            else:
                for skey, shift in [("Down", "down"), ("Up", "up")]:
                    plotting.ratioHistPlot(
                        **plot_params,
                        variation=(wshift, shift, wsamples),
                        title=f"{selection_regions_label[label]} Region {wlabel} Unc. {skey} Shapes",
                        name=f"{plot_dir}/{cutstr}{label}_region_bb_mass_{wshift}_{shift}.pdf",
                    )

        if pass_region:
            plotting.ratioHistPlot(
                **plot_params,
                sig_err="txbb",
                title=rf"{selection_regions_label[label]} Region $T_{{Xbb}}$ Shapes",
                name=f"{plot_dir}/{cutstr}{label}_region_bb_mass_txbb.pdf",
            )